In [7]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
#streaming은 model의 응답이 생성되는 것을 볼 수 있게 해준다. console에서의 응답 진행과정을 확인할 수 있음 

prompt.format(country="South Korea")


'What is the capital of South Korea?'